task https://stepik.org/lesson/701337/step/7?unit=701406

In [11]:
import requests
from bs4 import BeautifulSoup
import json

url = 'https://parsinger.ru/html/index1_page_1.html'
site = 'http://parsinger.ru/html/'
count = 0

def get_response(url):
    response = requests.get(url=url)
    response.encoding = 'utf-8'  
    return response

def list_categories(response,site):
    soup = BeautifulSoup(response.text, 'lxml')
    categories = [f"{site}{link['href']}" for link in soup.find('div', class_='nav_menu').find_all('a')]    
    return categories

def cook_pagination(response,site):
    soup = BeautifulSoup(response.text, 'lxml')
    pages = [f"{site}{link['href']}" for link in soup.find('div', class_='pagen').find_all('a')]        
    return pages


def cook_products(url, response, name_cat):
    soup = BeautifulSoup(response.text, 'lxml')
    categories = name_cat
    name = soup.find("p", id='p_header' ).text
    article = soup.find('p', class_='article').text.split()[1]
    availability = soup.find("span", id='in_stock').text    
    price = soup.find('span', id='price').text
    old_price = soup.find('span', id='old_price').text
    
    flat = categories, name, article, availability.split()[2], price, old_price, url
    return flat

for category in list_categories(get_response(url), site):
    for page in cook_pagination(get_response(category),site):
        response = get_response(page)
        soup = BeautifulSoup(response.text, 'lxml') 
        result_json = []
        name_category = soup.find('div', class_='nav_menu').find_all('div')
        name_cat = [x['id'] for x in name_category]
        url_products = [f"{site}{link['href']}" for link in soup.find_all('a', class_='name_item')]        
        
        for url_product in url_products:
            response = get_response(url_product)
            soup = BeautifulSoup(response.text, 'lxml')            
            list_item = cook_products(url_product, get_response(url_product), name_cat[count])
            li_id = [x['id'] for x in soup.find('ul', id='description').find_all('li')]
            temp = [x.text for x in soup.find_all('li')]
            description_value = [item.split(':', maxsplit=1)[1].strip() for item in temp]
            dict_description = dict(zip(li_id, description_value))
        
            result_json.append({
                'categories': [x.strip() for x in list_item][0],
                'name': [x.strip() for x in list_item][1],
                'article': [x.strip() for x in list_item][2],
                'description': dict_description,
                'count': [x.strip() for x in list_item][3],
                'price': [x.strip() for x in list_item][4],
                'old_price': [x.strip() for x in list_item][5],
                'link': [x.strip() for x in list_item][6]
                })
        
        with open('res10.json', 'a', encoding='utf-8') as file:
            json.dump(result_json, file, indent=4, ensure_ascii=False)
    count += 1

print('Файл res10.json создан')



Файл res10.json создан
